# 2020-01-22-numba-demo

## 1. This notebook

This demo of Awkward Array was presented on January 22, 2020, before the first stable version (1.0) was released. Some interfaces may have changed. To run this notebook, make sure you have version 0.1.87  ([GitHub](https://github.com/scikit-hep/awkward-1.0/releases/tag/0.1.87), [pip](https://pypi.org/project/awkward1/0.1.87/)) by installing

```bash
pip install 'awkward1==0.1.87'
```

before executing it in Jupyter (or include that release number in the Binder URL).

Depending on where you execute this notebook and how you installed or didn't install Awkward Array, you might need the following.

In [2]:
# The base of the GitHub repo is two levels up from this notebook.
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))

## 2. Introduction to Awkward Array

Awkward Array is a library for manipulating data structures with NumPy-like idioms. For a core set of NumPy features—slicing, broadcasting, array-at-a-time operations, and such—it is a strict generalization from rectilinear arrays of numeric data types to unequal-width and heterogeneous lists and nested objects.

The name arose organically: these kinds of arrays are usually awkward to deal with.

### 2.1 Distinction from NumPy object arrays

Although NumPy arrays can contain arbitrary objects with `dtype('O')` type, those arrays can't be sliced or operated on with NumPy's usual idioms because they're really just pointers to pure Python objects.

In [3]:
import numpy as np
import awkward1 as ak

nparray = np.array([[1, 2, 3], [], [4, None, 5], [{"something": 1, "else": [2, 3]}]])
akarray = ak.Array([[1, 2, 3], [], [4, None, 5], [{"something": 1, "else": [2, 3]}]])

In [4]:
# NumPy can't slice the substructure of Python objects
nparray[2:, 0]

IndexError: too many indices for array

In [5]:
# Awkward can slice this substructure
akarray[2:, 0]

<Array [4, {something: 1, else: [2, 3]}] type='2 * ?union[int64, {"something": i...'>

In [6]:
# NumPy can't pass ufuncs into Python objects
np.sin(nparray)

TypeError: loop of ufunc does not support argument 0 of type list which has no callable sin method

In [7]:
# Awkward can pass ufuncs all the way down to the numericl data
np.sin(akarray)

<Array [[0.841, 0.909, 0.141], ... 0.141]}]] type='4 * var * ?union[float64, {"s...'>

In [8]:
# Here's a little more detail on the above:
ak.tolist(np.sin(akarray))

[[0.8414709848078965, 0.9092974268256817, 0.1411200080598672],
 [],
 [-0.7568024953079282, None, -0.9589242746631385],
 [{'something': 0.8414709848078965,
   'else': [0.9092974268256817, 0.1411200080598672]}]]

### 2.2 Columnar structure

Like NumPy (as well as [Apache Arrow](https://arrow.apache.org/) and [XND](https://xnd.io/)), Awkward Array operates on columnar arrays and prefers _O(1)_ views, rather than _O(n)_ computations (where _n_ is the number of elements in the array) wherever possible.

In [9]:
# Columnar structure of the above array
akarray.layout

<ListOffsetArray64>
    <offsets><Index64 i="[0 3 3 6 7]" offset="0" at="0x59194d151080"/></offsets>
    <content><IndexedOptionArray64>
        <index><Index64 i="[0 1 2 3 -1 4 5]" offset="0" at="0x59194d1550a0"/></index>
        <content><UnionArray8_64>
            <content index="0">
                <NumpyArray format="l" shape="5" data="1 2 3 4 5" at="0x59194d153090"/>
            </content>
            <content index="1">
                <RecordArray>
                    <field index="0" key="something">
                        <NumpyArray format="l" shape="1" data="1" at="0x59194d1590c0"/>
                    </field>
                    <field index="1" key="else">
                        <ListOffsetArray64>
                            <offsets><Index64 i="[0 2]" offset="0" at="0x59194d15b0d0"/></offsets>
                            <content><NumpyArray format="l" shape="2" data="2 3" at="0x59194d15d0e0"/></content>
                        </ListOffsetArray64>
                 

We started printing layout object representations in Pythonic `<angle brackets>` until we had to start nesting them, then XML seemed like an obvious generalization.

High-level data types are expressed in [Datashape](https://datashape.readthedocs.io/en/latest/) notation:

In [10]:
ak.typeof(akarray)

4 * var * ?union[int64, {"something": int64, "else": var * int64}]

with [extensions where necessary](https://github.com/blaze/datashape/issues/237). Similarly, these arrays will be portable to and from Apache Arrow (and other formats, if requested).

The idea is that Awkward Array provides **manipulation** capabilities, not **serialization** or **transport**.

### 2.3 Relevance for Numba

Numba, as you know, provides **computation** capabilities in a way that complements NumPy. Whereas NumPy requires array-at-a-time operations for performance, Numba enables imperative, pure Python code to have equal and often exceeding performance.

The analogy with Awkward is one-to-one:

|   | without Numba | with Numba |
|:-:|:-------------:|:----------:|
| **with NumPy** | array-at-a-time processing on numbers | general code on NumPy arrays and Python objects |
| **with Awkward** | array-at-a-time processing on data structures | general code on Awkward data structures |

The Awkward Array library includes Numba extensions with near feature parity: most operations that run outside of JIT-compiled functions run inside them as well.

In [11]:
import numba as nb

@nb.jit(nopython=True)
def run(array):
    out = np.empty(len(array), np.float64)
    for i in range(len(array)):
        out[i] = array[i]["x"]
        for y in array[i]["y"]:
            out[i] += y
    return out

akarray = ak.Array([{"x": 100, "y": [1.1, 2.2]}, {"x": 200, "y": []}, {"x": 300, "y": [3.3]}])

# Works for the layout classes, but not the high-level ak.Array wrapper yet.
run(akarray.layout)

array([103.3, 200. , 303.3])

Although Numba can take and return builtin Python objects (e.g. tuples, lists, dicts) and can let you define extensions for class instances with `@jitclass`, these objects need to be unboxed and boxed to Python types, which can be a bottleneck for large datasets. (At the very least, Python objects are a memory bottleneck!)

Since data in an Awkward Array are columnar, boxing and unboxing scales with the depth of the columnar layout, not the number of elements in the array. In this example, 4 array nodes were unboxed, though the array could have a million elements.

In [12]:
akarray.layout

<RecordArray>
    <field index="0" key="x">
        <NumpyArray format="l" shape="3" data="100 200 300" at="0x59194d18aa40"/>
    </field>
    <field index="1" key="y">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 2 3]" offset="0" at="0x59194d18ca50"/></offsets>
            <content><NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x59194d18ea60"/></content>
        </ListOffsetArray64>
    </field>
</RecordArray>

<img src="img/example-hierarchy.png" style="width: 800px;">

In [13]:
%%timeit -n 1 -r 1

# Make an array of the same type with a million elements.
builder = ak.FillableArray()

for i in range(1000000):
    builder.beginrecord()
    builder.field("x")
    builder.integer(np.random.poisson(3) * 100)
    builder.field("y")
    builder.beginlist()
    for j in range(np.random.poisson(3)):
        builder.real(np.random.randint(5) * 1.1)
    builder.endlist()
    builder.endrecord()

akarray = builder.snapshot()
print(akarray, end="\n\n")

[{x: 300, y: [1.1, 3.3, 2.2]}, {x: 200, y: [2.2, ... {x: 400, y: [1.1, 0, 1.1]}]

29.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
run(akarray.layout)

array([103.3, 200. , 303.3])

We can even use Numba to build data structures with `FillableArray`, with a dramatic speedup (here, 50×).

In [15]:
%%timeit -n 1 -r 1

@nb.jit(nopython=True)
def build(builder):
    for i in range(1000000):
        builder.beginrecord()
        builder.field("x")
        builder.integer(np.random.poisson(3) * 100)
        builder.field("y")
        builder.beginlist()
        for j in range(np.random.poisson(3)):
            builder.real(np.random.randint(5) * 1.1)
        builder.endlist()
        builder.endrecord()
    return builder

print(ak.Array(build(ak.layout.FillableArray()).snapshot()), end="\n\n")

[{x: 400, y: [2.2, 1.1]}, {x: 400, y: [0, 0, ... {x: 400, y: [4.4, 3.3, 2.2, 0]}]

784 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


The equivalent in Numba is about as fast, though it has to box _O(million)_ lists and numbers.

In [16]:
%%timeit -n 1 -r 1

@nb.jit(nopython=True)
def build():
    outx = []
    outy = []
    for i in range(1000000):
        outx.append(np.random.poisson(3) * 100)
        tmp = []
        for j in range(np.random.poisson(3)):
            tmp.append(np.random.randint(5) * 1.1)
        outy.append(tmp)
    return (outx, outy)

outx, outy = build()
print(outx[:5])
print(outy[:5], end="\n\n")

[200, 200, 100, 400, 300]
[[2.2, 4.4, 2.2], [0.0, 0.0, 4.4], [4.4], [0.0, 0.0, 0.0], [2.2, 1.1, 4.4, 0.0, 1.1]]

1.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.4 Why particle physics?

In our field, big datasets and nested data structures are ubiquitous. Nearly every physics analysis has to associate undiffentiated final-state particle trajectories to a hypothetical, hierarchical decay chain, such as this one:

<img src="img/ttbarHDecayDiagram_expanded.png" style="width: 800px;">

for billions of collision events.

Once the match has been made, the labeled particles could be represented by a rectilinear table. However, the task of looping through candidate combinations and uncertainties associated with pruning candidates _is the whole analysis, not a preprocessing step_. For most of the analysis, we are working with unequal-sized collections of objects (momentum vector components, energy, and other variables derived from detector measurements).

Our field has always had this problem. Even before Fortran had objects (or a multi-line `IF` statement), specialized physics software added the ability to operate on data structures. This is an exerpt from [Initiation to Hydra (1974)](https://cds.cern.ch/record/864527) by R.K. Böck, describing the concept of a non-numerical data structure to a physics audience.

<img src="img/hydra-2.png" style="width: 500px;">

Since the 1990's, object-oriented programming in C++ has been good for our field: it's natural to think of each particle as a C++ object with statically typed attributes, collected in variable-length `std::vector<Particle>`s.

**However,**

   * **we want to use Python:** last year marked a crossover threshold in which more physicist's GitHub repositories were [written in Python than C++](img/github-fraction.png),
   * **we still have huge datasets:** 10's of TB after considerable reduction (from the original 100's of PB).

NumPy would be good for our analysis scripts if it had more data types than rectilinear arrays of numbers. Pandas's [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) is good for a single ragged dimension, but doesn't come close to what we need. Numba is excellent, but we need to get large datasets in and out efficiently. (Our [file format is already columnar](https://github.com/scikit-hep/uproot#readme); making intermediate Python objects would be a waste.)

Awkward Array was introduced to particle physicists in September 2019 and [is very popular in our community](../img/awkward-0-popularity.png). Emboldened by this response, I'm reimplementing it in a way that will make Numba, C++, and GPU integration easier to maintain.

### 2.5 Why not particle physics?

There's nothing domain-specific about nested data structures. The kinds of operations we want to do are [logical extensions of NumPy](https://github.com/jpivarski/2019-07-29-dpf-python/blob/master/03-columnar-data-analysis.ipynb) and can also be expressed as [per-array-item SQL](https://github.com/lgray/AwkwardQL#readme). Also wanting a general programming environment in Numba also has nothing, specifically, to do with particle physics.

There must be many other applications. How about this one?

In [18]:
!wget https://datahub.io/core/geo-countries/r/countries.geojson

--2020-01-21 16:24:22--  https://datahub.io/core/geo-countries/r/countries.geojson
Resolving datahub.io (datahub.io)... 104.24.113.103, 104.24.112.103, 2606:4700:3035::6818:7167, ...
Connecting to datahub.io (datahub.io)|104.24.113.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/geo-countries/countries/archive/23f420f929e0e09c39d916b8aaa166fb/countries.geojson [following]
--2020-01-21 16:24:24--  https://pkgstore.datahub.io/core/geo-countries/countries/archive/23f420f929e0e09c39d916b8aaa166fb/countries.geojson
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 104.24.113.103, 104.24.112.103, 2606:4700:3036::6818:7067, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.24.113.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24090863 (23M) [application/octet-stream]
Saving to: ‘countries.geojson’

countries.geojson   100%[===================>]  22.97M  3.01MB/s    in 13s     

In [19]:
countries = ak.Array("countries.geojson")
countries

<Array [... [30, -15.6], [30, -15.6]]]}}]}]] type='1 * var * {"type": string, "f...'>

In [20]:
ak.typeof(countries)

1 * var * {"type": string, "features": var * {"type": string, "properties": {"ADMIN": string, "ISO_A3": string}, "geometry": {"type": string, "coordinates": var * var * var * union[float64, var * float64]}}}

In [21]:
# longitude coordinates
countries["features", "geometry", "coordinates", :, :, :, :, :, 0]

<Array [... 29.8, 29.8, 29.9, 30, 30]]]]] type='1 * var * var * var * var * unio...'>

In [22]:
# latitude coordinates
countries["features", "geometry", "coordinates", :, :, :, :, :, 1]

<Array [... -15.6, -15.6, -15.6, -15.6]]]]] type='1 * var * var * var * var * un...'>

The fact that data analysts have managed so far with table-oriented tools like Pandas and SQL doesn't mean that they couldn't get more done if they had irregular data structures, too. Being able to write arbitrary algorithms in Numba on these big, irregular data structures further extends that reach.

## 3. Awkward Array implementations

### 3.1 Motivation for Awkward 1.0

The "0.x" version of Awkward (the one that is currently in use) is implemented entirely in NumPy. For performance, all of its algorithms must be expressed in a sequence of array-at-a-time calls, which led to [extremely clever case-by-case solutions](https://github.com/scikit-hep/awkward-array/blob/3442c51ed5dafb7d94f828c6cdc07659f9c03244/awkward/array/jagged.py#L1120-L1213).

In the end, though, we just can't generalize without being able to write `for` loops.

In [23]:
import awkward as oldak

oldarray = oldak.fromiter([[[0.0, 1.1, 2.2], [], [3.3, 4.4]], [[5.5]], [], [[6.6, 7.7, 8.8, 9.9]]])
newarray =       ak.Array([[[0.0, 1.1, 2.2], [], [3.3, 4.4]], [[5.5]], [], [[6.6, 7.7, 8.8, 9.9]]])

In [24]:
oldarray[:, ::-1, ::2]

NotImplementedError: this implementation cannot slice a JaggedArray in more than two dimensions

In [29]:
ak.tolist(newarray[:, ::-1, ::2])

[[[3.3], [], [0.0, 2.2]], [[5.5]], [], [[6.6, 8.8]]]

The original library had [awkward-numba](https://github.com/scikit-hep/awkward-array/tree/master/awkward-numba) and [awkward-cpp](https://github.com/scikit-hep/awkward-array/tree/master/awkward-cpp) subprojects, with the intention of adding an awkward-gpu, but the difficulty of maintaining them as independent implementations compounded.

### 3.2 Layered architecture

Awkward 1.0 is implemented in four layers:

   1. The `ak.Array` class and `ak.*` operations accessed by data analysts.
   2. The node objects that compose to form columnar data structures (in Python via pybind11).
   3. The C++ and Numba implementations of those nodes; reference-counted data structures that own memory.
   4. The CPU (and someday GPU) operations that navigate and fill arrays.

<img src="../img/awkward-1-0-layers.png" style="width: 500px;">

The spirit of NumPy—driving array-at-a-time operations from slow code with fast, precompiled kernels—is shifted down one layer: the C++ implementation is written without concern for speed, but only _O(1)_ operations are performed. The C++ is full of `std::shared_ptr` and `virtual` method calls, but _no loops over array data_ at this layer.

The Numba implementation, however, needs to be fast because Numba objects will be constructed in (the user's) loops over array data.

No memory allocation is performed in layer 4. It looks a lot like C code and has a pure C interface, so that C++ and Numba can both benefit from its implementations.

**A walk through the code:** how `__getitem__(tuple)` is implemented in

   * [old Awkward](https://github.com/scikit-hep/awkward-array/blob/master/awkward/array/jagged.py#L509-L779): lots of integer array tricks and `numpy.take`, valid for many special cases;
   * [new C++](https://github.com/scikit-hep/awkward-1.0/blob/master/src/libawkward/array/ListArray.cpp#L483-L620): recursive walk through the tuple, `dynamically_casting` class types and calling C kernels;
   * [new Numba](https://github.com/scikit-hep/awkward-1.0/blob/master/awkward1/_numba/array/listarray.py#L257-L490): recursive walk through the tuple, generating specialized code that calls C kernels;
   * [new C kernels](https://github.com/scikit-hep/awkward-1.0/blob/master/src/cpu-kernels/getitem.cpp#L288-L509): `for` loops on raw arrays.

### 3.3 Awkwardness everywhere

Since the `__getitem__(tuple)` operation is actually defined in the C kernels, C++ and Numba are calling the same code when they run it.

In [28]:
@nb.jit(nopython=True)
def demo(array):
    return array[:, ::-1, ::2]

ak.tolist(demo(newarray.layout))

[[[3.3], [], [0.0, 2.2]], [[5.5]], [], [[6.6, 8.8]]]

That way, implementations don't diverge.

Thanks to this reuse, we can also run any of these operations in C++.

In [64]:
open("test-program.cpp", "w").write("""

#include "awkward/Slice.h"
#include "awkward/io/json.h"
#include "awkward/array/ListOffsetArray.h"

namespace ak = awkward;

int main(int, char**) {
  std::shared_ptr<ak::Content> array = ak::FromJsonString(
      "[[[0.0, 1.1, 2.2], [], [3.3, 4.4]], [[5.5]], [], [[6.6, 7.7, 8.8, 9.9]]]",
      ak::FillableOptions(1024, 2.0));

  std::vector<std::shared_ptr<ak::SliceItem>> slice({
      std::make_shared<ak::SliceRange>(ak::Slice::none(), ak::Slice::none(), ak::Slice::none()),
      std::make_shared<ak::SliceRange>(ak::Slice::none(), ak::Slice::none(), -1),
      std::make_shared<ak::SliceRange>(ak::Slice::none(), ak::Slice::none(), 2)});

  std::shared_ptr<ak::Content> sliced = array->getitem(ak::Slice(slice));
  std::cout << sliced->tojson(false, 1) << std::endl;

  return 0;
}
""")

792

In [65]:
!g++ -I../../include -L../../awkward1 test-program.cpp -lawkward-static -lawkward-cpu-kernels-static -o test-program

In [66]:
!./test-program

[[[3.3],[],[0.0,2.2]],[[5.5]],[],[[6.6,8.8]]]


### 3.4 The high-level layer

At present, this is the least-developed part, but that's because writing Python is easy.  `:)`

The layout nodes are all mutually composable, but have been confusing to users in the old Awkward Array. Also, we want some high-level features to be persistent through slicing, and the easiest way to do this is to wrap the composable parts with their implementation details inside a non-composable "shell" called `ak.Array`.

<img src="img/example-hierarchy.png" style="width: 800px;">

In [67]:
akarray = ak.Array([{"x": 100, "y": [1.1, 2.2]}, {"x": 200, "y": []}, {"x": 300, "y": [3.3]}])
akarray

<Array [{x: 100, y: [1.1, 2.2, ... y: [3.3]}] type='3 * {"x": int64, "y": var * ...'>

In [69]:
akarray.layout

<RecordArray>
    <field index="0" key="x">
        <NumpyArray format="l" shape="3" data="100 200 300" at="0x59194ec0a690"/>
    </field>
    <field index="1" key="y">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 2 3]" offset="0" at="0x59194ecc82e0"/></offsets>
            <content><NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x59194f7fd220"/></content>
        </ListOffsetArray64>
    </field>
</RecordArray>

In [70]:
akarray.layout.field("y")

<ListOffsetArray64>
    <offsets><Index64 i="[0 2 2 3]" offset="0" at="0x59194ecc82e0"/></offsets>
    <content><NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x59194f7fd220"/></content>
</ListOffsetArray64>

In [71]:
akarray.layout.field("y").content

<NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x59194f7fd220"/>

In [72]:
np.asarray(akarray.layout.field("y").content)

array([1.1, 2.2, 3.3])

**HERE:** type overloading

### 3.5 The FillableArray builder

Unlike NumPy, Awkward Arrays are immutable. (They're too complicated for `__setitem__` to make sense.) As such, we need a way to make them, so `FillableArray` provides a builder pattern.

In [74]:
builder = ak.FillableArray()
                           # equivalent JSON
builder.beginlist()        # [
builder.integer(1)         #   1,
builder.integer(2)         #   2,
builder.integer(3)         #   3
builder.endlist()          # ]
builder.beginlist()        # [
builder.endlist()          # ]
builder.beginlist()        # [
builder.integer(4)         #   4,
builder.null()             #   null,
builder.integer(5)         #   5
builder.endlist()          # ]
builder.beginlist()        # [
builder.beginrecord()      #   {
builder.field("something") #     "something":
builder.integer(1)         #      1,
builder.field("else")      #      "else":
builder.beginlist()        #      [
builder.integer(2)         #        2,
builder.integer(3)         #        3
builder.endlist()          #      ]
builder.endrecord()        #   }
builder.endlist()          # ]

ak.tolist(builder.snapshot())

[[1, 2, 3], [], [4, None, 5], [{'something': 1, 'else': [2, 3]}]]

The type of the data depends on the order in which `FillableArray`'s methods are called; its data depends on the values passed. You can create Awkward Arrays in the same code that would otherwise have printed out JSON. It makes the nested layout nodes:

In [75]:
builder.snapshot().layout

<ListOffsetArray64>
    <offsets><Index64 i="[0 3 3 6 7]" offset="0" at="0x59194d481ad0"/></offsets>
    <content><IndexedOptionArray64>
        <index><Index64 i="[0 1 2 3 -1 4 5]" offset="0" at="0x59194ef3f380"/></index>
        <content><UnionArray8_64>
            <content index="0">
                <NumpyArray format="l" shape="5" data="1 2 3 4 5" at="0x59194d444b00"/>
            </content>
            <content index="1">
                <RecordArray>
                    <field index="0" key="something">
                        <NumpyArray format="l" shape="1" data="1" at="0x59194ef6aa30"/>
                    </field>
                    <field index="1" key="else">
                        <ListOffsetArray64>
                            <offsets><Index64 i="[0 2]" offset="0" at="0x59194e329370"/></offsets>
                            <content><NumpyArray format="l" shape="2" data="2 3" at="0x59194f8076a0"/></content>
                        </ListOffsetArray64>
                 

You can be quite free with it:

In [85]:
def deepnesting(builder, depth):
    if depth == 0:
        builder.integer(np.random.randint(0, 10))
    else:
        builder.beginlist()
        for j in range(np.random.poisson(3)):
            deepnesting(builder, depth - 1)
        builder.endlist()

builder = ak.FillableArray()
deepnesting(builder, 5)
ak.tolist(builder.snapshot())

[[[[[[7, 1, 1, 9, 8, 0], [9, 7, 6, 5, 8, 5], [4, 3]],
    [[2], [0, 6, 2]],
    [[2, 9], [7]],
    [[4, 5], [3, 0, 3]]],
   [[[8, 1], [5], [3, 2, 2, 1, 8, 4, 3], [0, 2, 0, 5]],
    [[6, 1, 9, 6], [6, 6, 2, 3], [2, 0]],
    [[1, 6]],
    [[6, 7], [8, 2]],
    [[8]]],
   [[[6, 3], [6, 7, 0, 4], [4, 7, 5, 4], [3, 7]],
    [[0, 2, 1, 2, 0], [5, 0]],
    [[6, 6, 3], [4, 1]]]],
  [],
  [[[[8, 1, 6, 9], [5, 0, 4, 1, 9, 0], [4, 5]],
    [[5], [4, 0, 3, 6, 2], [6, 2], [8, 7, 3]]],
   [[[2, 7, 5], [7, 3, 0, 2, 3, 3, 7]],
    [[2, 8, 2], [5, 8, 2], [3, 5]],
    [[8, 5], [3, 7, 9]]],
   [[[6, 4, 6], [], [7, 6], [2, 3]],
    [[], [8, 6, 2], [5, 9, 4]],
    [[8, 0], [7, 5, 2, 8], [7, 3], [1, 4]]]],
  [[[[2, 7]], [[1, 3, 1], [4], [6, 1, 6], [6, 2, 5, 6]], [[7]]]],
  [[[[1], [6, 5]],
    [[3, 3, 4, 7], [7, 0], [0]],
    [[3, 9, 4, 6, 4], [6, 8, 7], [3, 2]],
    [[2, 3], [2, 2, 9]]],
   [[[7, 6, 9]],
    [[0], [2, 2, 1]],
    [[3, 2, 1, 8, 6, 9], [9, 0, 9, 5], [5, 9, 9, 2, 4, 2], [0, 2, 0, 2, 8]],
    

Even in Numba:

In [88]:
@nb.jit(nopython=True)
def deepnesting(builder, depth):
    if depth == 0:
        builder.integer(np.random.randint(0, 10))
    else:
        builder.beginlist()
        for j in range(np.random.poisson(3)):
            deepnesting(builder, depth - 1)
        builder.endlist()

builder = ak.layout.FillableArray()
deepnesting(builder, 5)
ak.tolist(builder.snapshot())

[[[[[[6, 6, 2]],
    [[5, 7, 7, 4, 9, 8], [4, 5, 7, 5, 7], [6, 8, 3, 0, 4, 7]],
    [[3, 9, 4], [5, 7, 3, 1, 9, 9], [6, 5]],
    [[5], [3, 7]],
    [[4, 4, 9, 3, 4, 7], [5, 7, 9, 5, 8], [8, 1, 2, 9]]],
   [[[2, 2]],
    [[2, 1, 2, 2, 0, 0], [], [9, 3], [9], [8, 7]],
    [[8, 9, 2, 8, 3, 3], [9, 6], [6, 9, 0, 6, 8]]],
   [[[2, 8, 9, 0], [1, 3, 6, 8]],
    [[9, 3, 9, 2, 4], [3, 6]],
    [[1, 1], [8, 0, 4, 0], [3, 7, 1]],
    [],
    [[9, 4]]]]]]

In Numba, a `FillableArray` is an opaque type (entirely offloaded to C++, unlike all other array types).

In [89]:
nb.typeof(builder)

ak::FillableArrayType

## 4. The Numba implementation in depth

### 4.1 Layout node implementation example

Using [ListArray](https://github.com/scikit-hep/awkward-1.0/blob/master/awkward1/_numba/array/listarray.py) as an example, each layout node class has a corresponding Numba `Type` and `Model`.

`Types` carry as much information as Numba's lowering of NumPy arrays: number of dimensions and content type, but not length of each dimension.

```python
@numba.extending.typeof_impl.register(awkward1.layout.ListArray32)
@numba.extending.typeof_impl.register(awkward1.layout.ListArrayU32)
@numba.extending.typeof_impl.register(awkward1.layout.ListArray64)
def typeof(val, c):
    return ListArrayType(numba.typeof(numpy.asarray(val.starts)), numba.typeof(numpy.asarray(val.stops)), numba.typeof(val.content), numba.typeof(val.identities), util.dict2parameters(val.parameters))
```

Contents are part of the type specialization, recursively, so this information is included in the type name.

```python
class ListArrayType(content.ContentType):
    def __init__(self, startstpe, stopstpe, contenttpe, identitiestpe, parameters):
        assert startstpe == stopstpe
        assert isinstance(parameters, tuple)
        super(ListArrayType, self).__init__(name="ak::ListArray{0}{1}Type({2}, identities={3}, parameters={4})".format("" if startstpe.dtype.signed else "U", startstpe.dtype.bitwidth, contenttpe.name, identitiestpe.name, util.parameters2str(parameters)))
        self.startstpe = startstpe
        self.contenttpe = contenttpe
        self.identitiestpe = identitiestpe
        self.parameters = parameters
```

In the C++ implementation, I've introduced an array type for better reference counting. In Numba, I use Numba's lowered NumPy arrays.

```python
@numba.extending.register_model(ListArrayType)
class ListArrayModel(numba.datamodel.models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [("starts", fe_type.startstpe),
                   ("stops", fe_type.stopstpe),
                   ("content", fe_type.contenttpe)]
        if fe_type.identitiestpe != numba.none:
            members.append(("identities", fe_type.identitiestpe))
        super(ListArrayModel, self).__init__(dmm, fe_type, members)
```

Which means that some of my lowered implementations get to use Numba's functions. I use fully qualified function names everywhere: it's verbose, but helps a lot.

```python
@numba.extending.lower_builtin(operator.getitem, ListArrayType, numba.types.Integer)
def lower_getitem_int(context, builder, sig, args):
    rettpe, (tpe, wheretpe) = sig.return_type, sig.args
    val, whereval = args
    proxyin = numba.cgutils.create_struct_proxy(tpe)(context, builder, value=val)

    start = numba.targets.arrayobj.getitem_arraynd_intp(context, builder, tpe.startstpe.dtype(tpe.startstpe, wheretpe), (proxyin.starts, whereval))
    stop = numba.targets.arrayobj.getitem_arraynd_intp(context, builder, tpe.startstpe.dtype(tpe.stopstpe, wheretpe), (proxyin.stops, whereval))
    proxyslice = numba.cgutils.create_struct_proxy(numba.types.slice2_type)(context, builder)
    proxyslice.start = util.cast(context, builder, tpe.startstpe.dtype, numba.intp, start)
    proxyslice.stop = util.cast(context, builder, tpe.stopstpe.dtype, numba.intp, stop)
    proxyslice.step = context.get_constant(numba.intp, 1)

    outtpe = tpe.contenttpe.getitem_range()
    return tpe.contenttpe.lower_getitem_range(context, builder, outtpe(tpe.contenttpe, numba.types.slice2_type), (proxyin.content, proxyslice._getvalue()))
```

In some cases, I couldn't find an appropriate utility function, so I've written some of my own in [util.py](https://github.com/scikit-hep/awkward-1.0/blob/master/awkward1/_numba/util.py).

```python
def cast(context, builder, fromtpe, totpe, val):
    if isinstance(fromtpe, llvmlite.ir.types.IntType):
        if fromtpe.width == 8:
            fromtpe = numba.int8
        elif fromtpe.width == 16:
            fromtpe = numba.int16
        elif fromtpe.width == 32:
            fromtpe = numba.int32
        elif fromtpe.width == 64:
            fromtpe = numba.int64
        else:
            raise AssertionError("unrecognized bitwidth")
    if fromtpe.bitwidth < totpe.bitwidth:
        return builder.sext(val, context.get_value_type(totpe))
    elif fromtpe.bitwidth > totpe.bitwidth:
        return builder.trunc(val, context.get_value_type(totpe))
    else:
        return val
```

They're used, for example, when we need to call one of the C kernels.

```python
carrylength = numba.cgutils.alloca_once(builder, context.get_value_type(numba.int64))
util.call(context, builder, determine_carrylength,
    (carrylength,
     util.arrayptr(context, builder, arraytpe.startstpe, proxyin.starts),
     util.arrayptr(context, builder, arraytpe.stopstpe, proxyin.stops),
     lenstarts,
     context.get_constant(numba.int64, 0),
     context.get_constant(numba.int64, 0),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.start),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.stop),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.step)),
    "in {0}, indexing error".format(arraytpe.shortname))

nextoffsets = util.newindex(arraytpe.indexname, context, builder, numba.int64, builder.add(lenstarts, context.get_constant(numba.int64, 1)))
nextcarry = util.newindex64(context, builder, numba.int64, builder.load(carrylength))
util.call(context, builder, fill_carry,
    (util.arrayptr(context, builder, util.indextpe(arraytpe.indexname), nextoffsets),
     util.arrayptr(context, builder, util.index64tpe, nextcarry),
     util.arrayptr(context, builder, arraytpe.startstpe, proxyin.starts),
     util.arrayptr(context, builder, arraytpe.stopstpe, proxyin.stops),
     lenstarts,
     context.get_constant(numba.int64, 0),
     context.get_constant(numba.int64, 0),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.start),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.stop),
     util.cast(context, builder, numba.intp, numba.int64, proxyslicein.step)),
    "in {0}, indexing error".format(arraytpe.shortname))
```

We go through Numba's ctypes extension to call one fo the C kernels as an external function.

Yes, I know this means that user code compiled with Awkward Arrays in them can't be cached.

```python
def call(context, builder, fcn, args, errormessage=None):
    fcntpe = context.get_function_pointer_type(fcn.numbatpe)
    fcnval = context.add_dynamic_addr(builder, fcn.numbatpe.get_pointer(fcn), info=fcn.name)
    fcnptr = builder.bitcast(fcnval, fcntpe)

    err = context.call_function_pointer(builder, fcnptr, args)

    if fcn.restype is cpu.Error:
        assert errormessage is not None, "this function can return an error"
        proxyerr = numba.cgutils.create_struct_proxy(cpu.Error.numbatpe)(context, builder, value=err)
        with builder.if_then(builder.icmp_signed("!=", proxyerr.str, context.get_constant(numba.intp, 0)), likely=False):
            context.call_conv.return_user_exc(builder, ValueError, (errormessage,))
```

The `libawkward-cpu-kernels.so` is loaded in [cpu.py](https://github.com/scikit-hep/awkward-1.0/blob/master/awkward1/_numba/cpu.py). By design, these kernels have a very limited set of argument and return types.

```python
kernels = ctypes.cdll.LoadLibrary(libpath)

h2ctypes = {
    "bool": ctypes.c_uint8,
    "bool *": ctypes.POINTER(ctypes.c_uint8),
    "int8_t *": ctypes.POINTER(ctypes.c_int8),
    "const int8_t *": ctypes.POINTER(ctypes.c_int8),
    "uint8_t *": ctypes.POINTER(ctypes.c_uint8),
    "const uint8_t *": ctypes.POINTER(ctypes.c_uint8),
    "int32_t": ctypes.c_int32,
    "int32_t *": ctypes.POINTER(ctypes.c_int32),
    "const int32_t *": ctypes.POINTER(ctypes.c_int32),
    "uint32_t": ctypes.c_uint32,
    "uint32_t *": ctypes.POINTER(ctypes.c_uint32),
    "const uint32_t *": ctypes.POINTER(ctypes.c_uint32),
    "int64_t": ctypes.c_int64,
    "int64_t *": ctypes.POINTER(ctypes.c_int64),
    "const int64_t *": ctypes.POINTER(ctypes.c_int64),
    "Error": Error,
    "ERROR": Error,
    "void": None,
    }
```

Their signatures are read from [XML files generated by doxygen](https://github.com/scikit-hep/awkward-1.0/tree/master/awkward1/signatures) (so that I don't have to parse header files or depend on cffi.